In [505]:
import json
import os
import pandas as pd

Read english job ads

In [506]:
df = pd.read_json('/Users/cwestrup/thesis/data/sanoma/oikotie/jobs-en-2015-12-04T11:45:53.720971.json')

Select name and job_description

In [507]:
df = df.loc[:,['name','job_description']]
df.columns = ['title', 'content']
df.head()

,title,content
0,"Sandbox Software Engineer (ATP), Backend autom...",Coders needed<br />\nWe are looking for passio...
1,"Channel Manager, Cyber Security Services Nordics",F-Secure offers an exciting and challenging po...
2,Experienced Software Engineer in Test (ATP),Coders needed<br />\nWe’re looking for softwar...
3,"Experienced Software Engineer (ATP), Network S...",We’re looking for software engineers who can d...
4,Big Data & Machine Learning Software Engineer ...,We are looking for passionate Software Enginee...


Write out all jobs with names and full description text

In [508]:
df.index.name = 'ad_id'
df.to_csv('./data/alljobs.csv')

### Creating text chunks (paragraphs)

Select job descriptions

In [509]:
df_chunks = df.loc[:,['content']]

Replace '\n' by html br

In [510]:
to_spaces = '\n'
df_chunks = df_chunks.replace({'\n': '<br>'}, regex=True)

Replace html 'br' variants with 'BREAK'

In [511]:
to_paragraph_breaks = '(<br />)+|(<br/>)+|(<br>)+|(</br>)+'
df_chunks = df_chunks.replace(to_replace=to_paragraph_breaks, value='<BREAK>', regex=True)

Add break after html p tags

In [512]:
to_paragraph_breaks = '</p>|</ p>'
df_chunks = df_chunks.replace(to_replace=to_paragraph_breaks, value='</p><BREAK>', regex=True)

Remove all multiple occurences of BREAK tag

In [513]:
df_chunks = df_chunks.replace(to_replace='(<BREAK>)+', value='<BREAK>', regex=True)

Split rows by BREAK tags and stack up to a new DataFrame

In [514]:
df_chunks = df_chunks['content'].str.split('<BREAK>').apply(pd.Series)
df_chunks = df_chunks.stack().to_frame()

In [515]:
df_chunks = df_chunks.reset_index()

In [516]:
df_chunks.index.name = 'chunk_id'
df_chunks.columns = ['ad_id', 'chunk_count', 'content']

In [517]:
df_chunks.head(20)

,ad_id,chunk_count,content
chunk_id,,,
0,0,0,Coders needed
1,0,1,We are looking for passionate Software Enginee...
2,0,2,What’s in it for me?
3,0,3,"As part of the ATP team, you’ll be pioneering ..."
4,0,4,How to stand out as an applicant?
5,0,5,The following experience is an absolute must:
6,0,6,• Extensive prior experience with hypervisors ...
7,0,7,• We value prior experience integrating the ab...
8,0,8,"• Understanding of quality practices, continuo..."


In [518]:
df_chunks.to_csv('./data/chunks.csv')

Import into MongoDB with:

     mongoimport -d thesis -c ads --type csv --file ./data/alljobs.csv --headerline
     mongoimport -d thesis -c chunks --type csv --file ./data/chunks.csv --headerline